In [1]:
#This code has been adapted from: https://www.tensorflow.org/tutorials/generative/dcgan which creates artificial 28x28
#grayscale handwritten digits.
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import time
import os
import PIL
import pathlib
import tensorflow
tf.config.run_functions_eagerly(True)
from IPython import display
from PIL import Image
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.python.ops.numpy_ops import np_config
import glob

data_dir = "../input/simpsons-faces/cropped"
data_dir =pathlib.Path(data_dir)
img_height = 120
img_width = 120

images = glob.glob(str(data_dir)+"/*.png") #Retrieves images from data directory
images_size = len(images) #Needed later to calculate the duration of one epoch
images_size

In [2]:
import glob
import cv2
from random import shuffle

#Generator for loading images
#We used a data generator to preserve memory when this program is run
#We Converted the images from RGB color space to LAB so that the generator model only needs to fit on two dimensions(A,B)
#Whereas RGB would have requiered fitting on all 3. This improved performance and efficiency.
#The L in LAB represents the lightness of an image so it is practically just a grayscale image which preserves the 
#images structure allowing the generator model to have a much easier time producing images
def load_images(path, size=(120,120), batch = 64):    
    images = glob.glob(str(path)+"/*.png")
    shuffle(images)
    for image in images:
        img_list = []
        gray_list = []
        #These lists are effectively batches (Of size 64 in this case)
        for i in range(batch):
            loaded = cv2.imread(image)
            #Resize images so they fit into the Generator and Discriminator models
            loaded = cv2.resize(loaded, size, interpolation = cv2.INTER_AREA)
            img_list.append(cv2.cvtColor(loaded, cv2.COLOR_BGR2LAB)) #Default for cv2 is BGR so need to convert to LAB
            gray_list.append(cv2.cvtColor(loaded, cv2.COLOR_BGR2GRAY)) #Batch for grayscale images
        yield np.asarray(img_list)/255, np.expand_dims(np.asarray(gray_list),axis=3)/255
        #yield both batches and use expand dims so that grayscale output has dimensions 120,120,1 (Needed to fit the models)
        #Yield allows us to preserve memory

In [3]:
#Code to show LAB image and Grayscale image
#LAB can easily be converted to RGB so we will do this after the model trains on LAB images
import matplotlib.pyplot as plt
test = load_images(data_dir)
img, gray = next(test)
plt.imshow(img[0])
plt.show()
plt.imshow(gray[0], cmap="gray")
plt.show()

In [4]:
#Generator model borrowed from: https://github.com/OvaizAli/Image-Colorization-using-GANs
#Input: Grayscale image (120, 120, 1)
#Output: LAB image (120, 120, 3)
def make_generator_model():
    inputs = tf.keras.layers.Input( shape=( 120 , 120 , 1 ) )
    
    conv1 = tf.keras.layers.Conv2D( 16 , kernel_size=( 5 , 5 ) , strides=1)( inputs )
    conv1 = tf.keras.layers.LeakyReLU()( conv1 )
    conv1 = tf.keras.layers.Conv2D( 32 , kernel_size=( 3 , 3 ) , strides=1)( conv1 )
    conv1 = tf.keras.layers.LeakyReLU()( conv1 )
    conv1 = tf.keras.layers.Conv2D( 32 , kernel_size=( 3 , 3 ) , strides=1)( conv1 )
    conv1 = tf.keras.layers.LeakyReLU()( conv1 )

    conv2 = tf.keras.layers.Conv2D( 32 , kernel_size=( 5 , 5 ) , strides=1)( conv1 )
    conv2 = tf.keras.layers.LeakyReLU()( conv2 )
    conv2 = tf.keras.layers.Conv2D( 64 , kernel_size=( 3 , 3 ) , strides=1)( conv2 )
    conv2 = tf.keras.layers.LeakyReLU()( conv2 )
    conv2 = tf.keras.layers.Conv2D( 64 , kernel_size=( 3 , 3 ) , strides=1)( conv2 )
    conv2 = tf.keras.layers.LeakyReLU()( conv2 )

    conv3 = tf.keras.layers.Conv2D( 64 , kernel_size=( 5 , 5 ) , strides=1)( conv2 )
    conv3 = tf.keras.layers.LeakyReLU()( conv3 )
    conv3 = tf.keras.layers.Conv2D( 128 , kernel_size=( 3 , 3 ) , strides=1)( conv3 )
    conv3 = tf.keras.layers.LeakyReLU()( conv3 )
    conv3 = tf.keras.layers.Conv2D( 128 , kernel_size=( 3 , 3 ) , strides=1)( conv3 )
    conv3 = tf.keras.layers.LeakyReLU()( conv3 )

    bottleneck = tf.keras.layers.Conv2D( 128 , kernel_size=( 3 , 3 ) , strides=1 , activation='tanh' , padding='same' )( conv3 )

    concat_1 = tf.keras.layers.Concatenate()( [ bottleneck , conv3 ] )
    conv_up_3 = tf.keras.layers.Conv2DTranspose( 128 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' )( concat_1 )
    conv_up_3 = tf.keras.layers.Conv2DTranspose( 128 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' )( conv_up_3 )
    conv_up_3 = tf.keras.layers.Conv2DTranspose( 64 , kernel_size=( 5 , 5 ) , strides=1 , activation='relu' )( conv_up_3 )

    concat_2 = tf.keras.layers.Concatenate()( [ conv_up_3 , conv2 ] )
    conv_up_2 = tf.keras.layers.Conv2DTranspose( 64 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' )( concat_2 )
    conv_up_2 = tf.keras.layers.Conv2DTranspose( 64 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' )( conv_up_2 )
    conv_up_2 = tf.keras.layers.Conv2DTranspose( 32 , kernel_size=( 5 , 5 ) , strides=1 , activation='relu' )( conv_up_2 )

    concat_3 = tf.keras.layers.Concatenate()( [ conv_up_2 , conv1 ] )
    conv_up_1 = tf.keras.layers.Conv2DTranspose( 32 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu')( concat_3 )
    conv_up_1 = tf.keras.layers.Conv2DTranspose( 32 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu')( conv_up_1 )
    conv_up_1 = tf.keras.layers.Conv2DTranspose( 3 , kernel_size=( 5 , 5 ) , strides=1 , activation='sigmoid')( conv_up_1 )
    model = tf.keras.models.Model( inputs , conv_up_1 )
    return model

In [5]:
#Create generator model
make_generator_model()

In [6]:
#See what untrained generator produces when fed a random grayscale image
#Mostly gray because sigmoid function returns numbers in range (0,1) so many numbers lie around 0.5.
generator = make_generator_model()
generated_image = generator(np.expand_dims(gray[0], axis=0), training=False) #need expand dims to have batch size
plt.imshow(generated_image[0])

In [7]:
#GDiscriminator model borrowed from: https://github.com/OvaizAli/Image-Colorization-using-GANs
#Input: LAB image (120, 120, 3)
#Output: A number (positive or negative)
#The model will be trained to output positive values for real colorized images, and negative values for fake colorized images.
from keras.models import Model
from keras.layers import Conv2D, BatchNormalization, Activation, Dropout, Flatten, Dense, Input, LeakyReLU, Conv2DTranspose,AveragePooling2D
def make_discriminator_model():
    model = Sequential()
    model.add(Conv2D(32,(3,3), padding='same',strides=2,input_shape=(120,120,3)))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.25))

    model.add(Conv2D(64,(3,3),padding='same',strides=2))
    model.add(BatchNormalization())
    model.add(LeakyReLU(.2))
    model.add(Dropout(0.25))


    model.add(Conv2D(128,(3,3), padding='same', strides=2))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.25))


    model.add(Conv2D(256,(3,3), padding='same',strides=2))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.25))


    model.add(Flatten())
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    image = Input(shape=(120,120,3))
    validity = model(image)
    return Model(image,validity)

In [8]:
#See what happens when untrained discriminator is fed a generated "fake" colorized image
discriminator = make_discriminator_model()
decision = discriminator(generated_image)
print(decision)

In [9]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy()

In [10]:
#Loss function for the discriminator model
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output) - tf.random.uniform( shape=real_output.shape , maxval=0.1 ) , real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output) + tf.random.uniform( shape=fake_output.shape , maxval=0.1  ) , fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [11]:
#Loss function for the generator model
mse = tf.keras.losses.MeanSquaredError()
def generator_loss(fake_output , real_y):
    real_y = tf.cast( real_y , 'float32' )
    return mse( fake_output , real_y )

In [12]:
#Both models use the ADAM optimization algorithm
generator_optimizer = tf.keras.optimizers.Adam(0.0001)
discriminator_optimizer = tf.keras.optimizers.Adam(0.0001)

In [13]:
#Used to save and restore models, which can be helpful in case a long running training task is interrupted.
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [14]:
#Initialize variables used later on
EPOCHS = 150
BATCH_SIZE = 64

In [15]:
gen_loss_list = []
disc_loss_list = []
#Used to plot loss later on

In [16]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(input_x, real_y):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(input_x, training=True)
        real_output = discriminator(real_y, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(generated_images, real_y)
        disc_loss = discriminator_loss(real_output, fake_output)
        gen_loss_list.append(gen_loss.numpy())
        disc_loss_list.append(disc_loss.numpy())
    
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
generator.compile(
    optimizer=generator_optimizer,
    loss=generator_loss,
    metrics=['accuracy']
)

discriminator.compile(
    optimizer=discriminator_optimizer,
    loss=discriminator_loss,
    metrics=['accuracy']
)

In [17]:
#using steps instead of epochs because generator does not know when it has traversed one batch
#So instead we calculate epoch number ourselves and break the loop when the correct number of epochs has been reached
#1 epoch = images_size/BATCH_SIZE
def train(dataset, epochs):
    total_steps = 0 #Used to track how many times the train_step function has been called
    epoch = 0
    start = time.time() #Used to print how long each epoch takes to run
    for (y, x) in dataset:
        total_steps += 1
        if total_steps % (images_size//BATCH_SIZE) == 0: #If one epoch has passed
            display.clear_output(wait=True) #Clear window
            epoch+=1
            generate_and_save_images(generator, epoch + 1, gray[0:1]) #Save images for GIF later
            print ('Time for epoch {} is {} sec'.format(epoch, time.time()-start))
            start = time.time() #Used to print how long each epoch takes to run 
        if epoch >= epochs: #Break loop when total number of epochs have executed
            break
        train_step(x,y)
    
    # Generate Image after the final epoch
    display.clear_output(wait=True)
    generate_and_save_images(generator, epoch + 1, gray[0:1])#change to numpy to dim...
    print ('Time for epoch {} is {} sec'.format(epoch, time.time()-start))
    display.clear_output(wait=True)
    generate_and_save_images(generator, epoch, gray[0:1])

In [18]:
def generate_and_save_images(model, epoch, test_input):
    print("Image Generated")
    predictions = model(test_input, training=False) #change later to np to dim...
    prediction = predictions[0]*255 #bring to 0-255 range (Was in [0,1] range because sigmoid was the activation function)
    prediction = np.uint8(prediction)
    prediction = cv2.cvtColor(prediction, cv2.COLOR_LAB2RGB)
    prediction = prediction #normalize the image
    plt.imshow(prediction)
    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()
    print("Real")
    actual = np.uint8(img[0]*255)
    actual = cv2.cvtColor(actual, cv2.COLOR_LAB2RGB)
    plt.imshow(actual)
    plt.show()

In [19]:
train(load_images(data_dir), EPOCHS)

In [ ]:
#Plot both loss functions over a period of train steps (Using smoothing)
from scipy.interpolate import make_interp_spline
x = list(range(0, len(gen_loss_list)))
y = gen_loss_list
X_Y_Spline = make_interp_spline(x, y)
X_ = np.linspace(np.min(x), np.max(x), 300)
Y_ = X_Y_Spline(X_)
plt.title('Generator Loss')
plt.xlabel('Step Number')
plt.ylabel('Loss')
plt.plot(X_, Y_)
plt.show()

x = list(range(0, len(disc_loss_list)))
y = disc_loss_list
X_Y_Spline = make_interp_spline(x, y)
X_ = np.linspace(np.min(x), np.max(x), 500)
Y_ = X_Y_Spline(X_)
plt.title('Discriminator Loss')
plt.xlabel('Step Number')
plt.ylabel('Loss')
plt.plot(X_, Y_)
plt.show()

In [20]:
#Plot exmaple grayscale, colorized, and real version of exmaple character on test dataset
data_dir = "../input/simpsons-vs-real-faces/simpsons_vs_real_dataset/testB"
test = load_images(data_dir)
img, gray = next(test)
print("Grayscale")
plt.imshow(gray[0], cmap="gray")
plt.show()
generate_and_save_images(generator, EPOCHS, gray[0:1])

In [21]:
# Display a single image using the epoch number
def display_image(epoch_no):
    return PIL.Image.open('image_at_epoch_{:04d}.png'.format(epoch_no-1))
display_image(EPOCHS)

In [22]:
#used to create a GIF of the learning process through epochs
import imageio
import glob
anim_file = 'colorize.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
    filenames = glob.glob('image*.png')
    filenames = sorted(filenames)
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
    image = imageio.imread(filename)
    writer.append_data(image)

In [25]:
#Need to run: pip install git+https://github.com/tensorflow/docs
import tensorflow_docs.vis.embed as embed
embed.embed_file(anim_file)